# Quote API - Engine Documentation

Tasks to be completed:
1. Pull HTML results from stock query.
2. Parse financial information from the HTML results.
3. Export information in an organized way.
4. Package functions neatly for use in other projects.


## <a name="TOC"></a> Table of Contents:
---
1. [Proof of Concept](#proof)
2. [Fundamental Functions](#func)
3. [RESTful Functions](#REST)



In [30]:
# ------------------------- CONFIGURE ENVIRONMENT ------------------------- #

# Environment hard reset
%reset -f

# Libraries for scraping
import requests
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import Request, urlopen
import urllib.request
import urllib.parse
import urllib.error
import ssl
import ast
import os

# JSON Support
import json

# Configure paths
from pathlib import Path
profiles_path = Path('profiles/')


## <a name="proof"></a> [Proof of Concept](#TOC)
---

This section is built to demonstrate how the API could form queries for specific tickers and get the HTML results.


In [21]:
# ------------------------- FORM QUERY ------------------------- #

ticker = "HD"
query = f"https://finance.yahoo.com/quote/{ticker}"


# ------------------------- PARSE QUERY ------------------------- #

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Making the website believe that you are accessing it using a Mozilla browser
req = Request(query, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Creating a BeautifulSoup object of the HTML page for easy extraction of data.
soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
profile = {}
trading = {}
fundamentals = {}

# TRADING

# Previous Close
for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
    trading['Previous Close'] = td.text.strip()

# Open Value
for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
    trading['Open'] = td.text.strip()

# Bid
for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
    trading['Bid'] = td.text.strip()

# Ask
for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
    trading['Ask'] = td.text.strip()

# Day's Range
for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
    trading['Day Range'] = td.text.strip()

# Fifty-two Week Range
for td in soup.findAll('td', attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
    trading['Fifty-Two Week Range'] = span.text.strip()

# Trading Volume
for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
    trading['Day Volume'] = td.text.strip()

# Average 3M Volume
for td in soup.findAll('td', attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'}):
    trading['Average 3M Volume'] = td.text.strip()

# FUNDAMENTALS

# Market Capitalization
for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
    fundamentals['Market Capitalization'] = td.text.strip()

# Beta 3Y
for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
    fundamentals['Beta 3Y'] = td.text.strip()

# PE Ratio
for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
    fundamentals['PE Ratio'] = td.text.strip()

# EPS Ratio
for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
    fundamentals['EPS Ratio'] = td.text.strip()

# Earnings Date
for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'}):
    fundamentals['Earnings Date'] = td.text.strip()

# Dividend and Yield
for td in soup.findAll('td', attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
    fundamentals['Dividend'] = td.text.strip().split()[0]
    fundamentals['Dividend Yield'] = td.text.strip().split()[1].translate({ord(i): None for i in '()%'})

# Ex Dividend Date
for td in soup.findAll('td', attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
    fundamentals['Ex Dividend Rate'] = td.text.strip()

# One Year Target Price
for td in soup.findAll('td', attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'}):
    fundamentals['One Year Target Price'] = td.text.strip()

# Other Details
profile['Trading'] = trading
profile['Fundamental'] = fundamentals

# Other Details
profile


{'Trading': {'Previous Close': '347.94',
  'Open': '348.39',
  'Bid': '345.50 x 1000',
  'Ask': '344.80 x 800',
  'Day Range': '344.10 - 350.60',
  'Fifty-Two Week Range': 'Feb 22, 2022',
  'Day Volume': '5,342,363',
  'Average 3M Volume': '4,210,400'},
 'Fundamental': {'Market Capitalization': '362.216B',
  'PE Ratio': '23.19',
  'EPS Ratio': '14.95',
  'Earnings Date': 'Feb 22, 2022',
  'Dividend': '6.60',
  'Dividend Yield': '1.90',
  'Ex Dividend Rate': 'Dec 01, 2021',
  'One Year Target Price': '417.23'}}

## <a name="func"></a> [Fundamental Functions](#TOC)
---

Using this experimental code above we can develop functions with better error handling and minimalism.


In [31]:
# ------------------------- FORM QUERY ------------------------- #

def form_query(ticker):
    """
    Takes a stock ticker as input and returns the full HTTP request
    for the yahoo finance query.
    """
    return f"https://finance.yahoo.com/quote/{ticker}"


# ------------------------- READ HTML ------------------------- #

def read_html(query):
    """
    Uses a query and returns the prettified HTML for reading.
    """
    response = requests.get(query)
    soup = BeautifulSoup(response.text, 'lxml')
    print(soup.prettify())


# ----------------------- REQUEST WEBPAGE ---------------------- #

def request_webpage(query):
    """
    Sends request against Yahoo Finance API using the provided
    query. Returns the raw webpage bytes.
    """

    # For ignoring SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    # Making the website believe that you are accessing it using a Mozilla browser
    req = Request(query, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    return webpage


# ------------------------ PARSE WEBPAGE ------------------------ #

def parse_webpage(webpage):
    """
    Uses an HTML tree and searches for specific tags which house the
    stock information. This function utilizes try-catch methods
    for error handling of each data element. Returns empty for the
    missing values. The final return is a python dictionary.
    """

    # Creating a BeautifulSoup object of the HTML page for easy extraction of data.
    soup = BeautifulSoup(webpage, 'html.parser')
    profile = {}
    trading = {}
    fundamentals = {}

    # TRADING

    # Previous Close
    for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
        trading['Previous Close'] = td.text.strip()

    # Open Value
    for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
        trading['Open'] = td.text.strip()

    # Bid
    for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
        trading['Bid'] = td.text.strip()

    # Ask
    for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
        trading['Ask'] = td.text.strip()

    # Day's Range
    for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
        trading['Day Range'] = td.text.strip()

    # Fifty-two Week Range
    for td in soup.findAll('td', attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
        trading['Fifty-Two Week Range'] = td.text.strip()

    # Trading Volume
    for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
        trading['Day Volume'] = td.text.strip()

    # Average 3M Volume
    for td in soup.findAll('td', attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'}):
        trading['Average 3M Volume'] = td.text.strip()

    # FUNDAMENTALS

    # Market Capitalization
    for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
        fundamentals['Market Capitalization'] = td.text.strip()

    # Beta 3Y
    for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
        fundamentals['Beta 3Y'] = td.text.strip()

    # PE Ratio
    for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
        fundamentals['PE Ratio'] = td.text.strip()

    # EPS Ratio
    for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
        fundamentals['EPS Ratio'] = td.text.strip()

    # Earnings Date
    for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'}):
        fundamentals['Earnings Date'] = td.text.strip()

    # Dividend and Yield
    for td in soup.findAll('td', attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
        fundamentals['Dividend'] = td.text.strip().split()[0]
        fundamentals['Dividend Yield'] = td.text.strip().split()[1].translate({ord(i): None for i in '()%'})

    # Ex Dividend Date
    for td in soup.findAll('td', attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
        fundamentals['Ex Dividend Rate'] = td.text.strip()

    # One Year Target Price
    for td in soup.findAll('td', attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'}):
        fundamentals['One Year Target Price'] = td.text.strip()

    # Other Details
    profile['Trading'] = trading
    profile['Fundamental'] = fundamentals

    # Other Details
    return profile


# ----------------------- EXPORT PROFILE ----------------------- #

def export_profile(profiles_path, profile, ticker):
    """
    Exports a stock profile into the profiles folder as JSON.
    """
    file = profiles_path / f"{ticker}.json"
    with file.open("w") as fp:
        json.dump(profile, fp)


# ----------------------- IMPORT PROFILE ----------------------- #

def import_profile(profiles_path, ticker):
    """
    Imports a stock profile from the profiles folder.
    """
    file = profiles_path / f"{ticker}.json"
    raw = open(file).read()
    return json.loads(raw)


In [35]:
# ------------------------- TEST FUNCTIONS ------------------------- #

# Set test ticker
ticker = "HD"

# Form finance.yahoo query
query = form_query(ticker)

# Call for profile
webpage = request_webpage(query)
profile = parse_webpage(webpage)

# Export profile as JSON
export_profile(profiles_path, profile, ticker)

# Return query results
profile

{'Trading': {'Previous Close': '347.94',
  'Open': '348.39',
  'Bid': '345.50 x 1000',
  'Ask': '344.80 x 800',
  'Day Range': '344.10 - 350.60',
  'Fifty-Two Week Range': '246.59 - 420.61',
  'Day Volume': '5,342,363',
  'Average 3M Volume': '4,210,400'},
 'Fundamental': {'Market Capitalization': '362.216B',
  'PE Ratio': '23.19',
  'EPS Ratio': '14.95',
  'Earnings Date': 'Feb 22, 2022',
  'Dividend': '6.60',
  'Dividend Yield': '1.90',
  'Ex Dividend Rate': 'Dec 01, 2021',
  'One Year Target Price': '417.23'}}

## <a name="REST"></a> [RESTful Functions](#TOC)
---

Combining these basic functions together we can form the fundamentals of a REST API: CREATE, UPDATE, GET, DELETE. Some of these functions are redundant. For instance, the CREATE and UPDATE calls are the same as they both prefer to over-write existing information. This is intentional as it ensures that the profile is as up to date as possible but it might not be the best practice for API design as is makes the over-write decision for the user.


In [47]:
# ------------------------- CREATE PROFILE ------------------------- #
# Scrapes the information from finance.yahoo and updates the JSON
#   stock profile or creates the profile if it did not already exist
#   in the database.
#

def CreateProfile(ticker):
    
    # Form finance.yahoo query
    query = FormQuery(ticker)

    # Call for profile
    profile = ParseHTML(query)

    # Export profile as JSON
    ExportJSON(profile, ticker)
    
    # Return query results
    return json.loads(json.dumps(profile))


# ------------------------- UPDATE PROFILE ------------------------- #
# Creating and updating a profile are ultimately the same function as
#   they both overwrite the existing profile or create it if it does
#   not exist. For this reason, update calls just function-forward to
#   the create call.
#

def UpdateProfile(ticker):
    CreateProfile(ticker)
    
    
# ------------------------- GET PROFILE ------------------------- #
# This function finds the profile within the database and loads the
#   function as a JSON. In verbose mode, this function will also
#   return the dictionary version of the JSON file. In normal mode
#   it returns the JSON string itself, assuming an API application.
#

def GetProfile(ticker):
    return ImportJSON(ticker)



In [48]:
ticker = "LOW"
CreateProfile(ticker)

{'Trading': {'Previous Close': '106.50',
  'Open': '107.19',
  'Present Value': '106.40',
  'Bid': '106.40 x 900',
  'Ask': '106.45 x 800',
  'Day Volume': '1,545,339',
  'Average 3M Volume': '4,671,632',
  'Earnings Date': []},
 'Fundamental': {'Market Capitalization': '82.119B',
  'Beta 3Y': '1.42',
  'PE Ratio': '33.64',
  'EPS Ratio': '3.16',
  'Earnings Date': '20 Nov 2019',
  'Dividend': '2.20',
  'Dividend Yield': '2.07',
  'Ex Dividend Rate': '2019-10-22',
  'One Year Target Price': '121.62'}}

*Written by Alice Seaborn on 10/08/2019.*